In [ ]:
import pandas as pd
import numpy as np

#Format misc
import json
import os
import pickle

#Plot
import seaborn as sns
import pylab as plt

#Scrape
import statsmodels.formula.api as smf
import grequests
import requests
import bs4 as bs

#Detect language
import sys
sys.path.append("./")

#Combine sets and calculate diversity
from scipy.stats import entropy
import itertools
from collections import Counter

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))



%matplotlib inline

In [ ]:

netloc2freqs = pickle.load(open("./data/netloc2freqs.dump","rb+"))
people = pd.read_csv("data_clean/all_people_hap.csv",sep="\t")



In [ ]:
import itertools

def mkunion(*args):
    args = [hash2links[_] for _ in args[0].values]
    all_links = list(Counter(list(itertools.chain(*args))).values())
    return entropy(all_links)/np.log2(len(all_links))

p = people.groupby(["const_i","keyword","login_status","search_type"]).agg({"plugin_id":len,"result_hash":mkunion}).reset_index()
p["const_i"] = p["const_i"].astype(int)
#p.to_csv("data_clean/groupped_all_people.csv",sep="\t",index=None)
p = p.loc[p["plugin_id"]>0] #count of results
p.head(2)

In [ ]:
p = p.loc[p["plugin_id"]>1] #count of results
p["plugin_id"] = np.log10(p["plugin_id"])

parties = ['AfD',      'Bündnis90/Die Grünen', 'CDU', 'CSU',  'Die Linke',  'FDP', 'SPD']
p["person"] = p["keyword"].apply(lambda x: x in parties)

palette = ((255/255,174/255,143/255),(111/255,158/255,206/255))

In [ ]:
ress = []
for a,b in p.groupby("search_type"):
    print(a)
    for d,h in b.groupby(["keyword"]):
        print(d)
        plt.figure(figsize=(5,3.5))
        for j,h2 in h.groupby("login_status"):
            
            if not j:
                color=palette[0]
            else:
                color=palette[1]
            sns.regplot("plugin_id","result_hash",data=h2,truncate=True,
                        scatter_kws={"s":10,"alpha":0.5},lowess=True,color=color)#,y_partial="plugin_id")         
            
                
            plt.scatter([0,0],[0,0],label=str(j),color=color)
                
            plt.xlabel("Log10 number of people")
            plt.ylabel("Entropy")
            
            
            ress.append((a,d,h2))
            

#             sns.regplot("plugin_id","result_hash",data=h2,label=str(j),truncate=True,
#                         scatter_kws={"s":5,"alpha":0.5},y_partial="plugin_id")
#             slope, intercept = np.polyfit(h2["plugin_id"], h2["result_hash"],1)
#             h2["new_hash"] = h2["result_hash"]+slope*h2["plugin_id"]
#             sns.regplot("plugin_id","new_hash",data=h2,label=str(j)+"D",truncate=True,
#                         scatter_kws={"s":5,"alpha":0.5},y_partial="plugin_id")            
#             print(slope,intercept)
#             print(smf.ols("result_hash~plugin_id",data=h2).fit().summary())
#             continue
#         d()
            
        plt.xlim(1,3)
        if a == "search":
            plt.ylim(0.45,0.65)
        else:
            plt.ylim(0.55,0.7)
        plt.legend()
        
        if d == "Alice Weidel":
            sns.despine()
            plt.grid()
            
            legend = plt.legend(title="Login status:",fontsize=14)
            
            plt.setp(legend.get_title(),fontsize='small')
            plt.tight_layout()
            plt.savefig("figures/alice_{}.pdf".format(a))
            plt.show()
            
            
        else:
            plt.clf()
        



In [ ]:
#Ideology between parties

person2party_code = {'Alexander Gauland': 'AfD',
 'Alice Weidel': 'AfD',
 'Angela Merkel': 'CDU',
 'Cem Özdemir': 'Grüne',
 'Christian Lindner': 'FDP',
 'Dietmar Bartsch': 'Linke',
 'Katrin Göring-Eckardt': 'Grüne',
 'Martin Schulz': 'SPD',
 'Sahra Wagenknecht': 'Linke',
                    'Bündnis90/Die Grünen': 'Grüne',
                    'Die Linke': "Linke"}


p_i = people.groupby(["const_i","keyword","login_status","search_type"]).mean().reset_index()

sns.clustermap(people.loc[people["search_type"]=="news",["{}_ideo".format(party) for party in ["AfD","CDU","CSU","FDP","Grüne","Linke","SPD"]]].fillna(0).corr(),
           cmap="YlOrBr")

## Political personalization

In [ ]:
def shuffle(df, n=1, axis=0):     
    df = df.copy().reset_index()
    del df["index"]
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
        return df

In [ ]:
final1 = []
final1_null = []
for l,data in people.loc[(people["person"]==0)&(people["count_ideo_all"]>0)].groupby(["login_status","search_type"]):
    for party in ["AfD","Linke","FDP","Grüne","CDU","CSU","SPD"]:
        vals = "diff_{}_ideo".format(party)
        #vals = "diff_std_haps"
        data1 = data.loc[:,[vals,"const_i","keyword","user"]]
#         cand = party2person.get(party)
#         if cand is None:
#             continue
#         data1 = data1.loc[data1["keyword"].isin(cand)]
        all_keyword = pd.pivot_table(data1,values=vals,columns="keyword",index=["const_i","user"]).replace([0,1],np.NaN)
        persons = list(all_keyword.columns)
        all_keyword = all_keyword.reset_index()
        corr = []
        corr_null = []
        for k in range(len(persons)):
            for j in range(k+1,len(persons)):
                t = all_keyword[["const_i",persons[k],persons[j]]].dropna()
                #print(len(t))
                #if len(t) > 1000:
                c,p = pearsonr(t[persons[k]],t[persons[j]])
                corr.append(c)
                
                
                null = pd.concat([shuffle(d) for x,d in t.groupby(["const_i"])])
                c_null,p = pearsonr(null[persons[k]],null[persons[j]])
                corr_null.append(c_null)
        
                final1.append(list(l)+[party]+corr)
                final1_null.append(list(l)+[party]+corr_null)
        #if party == "AfD":
        #if l[1] == "news":
        print(party,l,np.mean(corr),np.mean(corr_null))
            #display((corr-np.identity(len(corr))).mean().reset_index().set_index("keyword").transpose())
        

In [ ]:
plt.figure(figsize=(4,3.5))

plt.subplot(1,1,1)
x = pd.DataFrame(final1)
x = pd.melt(x,id_vars=[0,1,2])

sns.barplot(x=1,y="value",hue=0,data=x,palette=palette,edgecolor="none")



sns.despine()
plt.xlabel("")
plt.ylabel("Average correlation between\n the ideology of results")
plt.xticks([0.,1.],["Google\n News","Google\n Search"])
plt.ylim(-0.0,0.14)
legend = plt.legend(fontsize=12)
legend.set_title("Login status:", prop = {'size':'small'})
# # plt.title("(A) Data")
# plt.tight_layout()

# plt.subplot(1,2,2)
# x = pd.DataFrame(final1_null)
# x = pd.melt(x,id_vars=[0,1,2])

# sns.barplot(x=1,y="value",hue=0,data=x,palette=palette,edgecolor="none")
# legend = plt.legend(title="Logged in:",fontsize=14)


# sns.despine()
# plt.xlabel("")
# plt.ylabel("Average correlation between\n ideology of links")

# plt.ylim(-0.0,0.14)
# plt.title("(B) Null model")
plt.tight_layout()


plt.savefig("figures/corr_ideo.pdf".format(a))
plt.show()

In [ ]:
x = pd.DataFrame(final1)
x = pd.melt(x,id_vars=[0,1,2,3])
plt.figure(figsize=(10,4))
sns.barplot(x=2,y="value",hue=0,data=x.loc[x[1]=="news"])
plt.figure(figsize=(10,4))
sns.barplot(x=2,y="value",hue=0,data=x.loc[x[1]=="search"])

# Polarization

In [ ]:
media = np.array([0.8,0.2])
own_op = 1
p = 0.
## Model

def update(x,w=None):
    #return (own_op*x+x**b*np.mean(media))/(own_op+x**b*np.mean(media) + (1-x)**b*(1-np.mean(media)))
    
#     diffs = np.abs(media-x)
#     f = np.abs(np.diff(diffs))[0]*slope
#     w = np.array([0.5,0.5])
#     w[diffs==np.min(diffs)] += f
#     w[diffs==np.max(diffs)] -= f
    if w is None:
        w = 1-np.abs(x-media)
        w = w/np.sum(w)
#    w = 0.5
    
    #Linear increase
    return (own_op*x+x**(b+p)*np.sum(media*w))/(own_op+x**(b+p)*np.sum(media*w) + (1-x)**(b+p)*(1-np.sum(media*w)))
 
plt.figure(figsize=(10,3.5))

plt.subplot(1,2,1)
for b in [0.6,0.7,0.8,0.9,1,1.1]:# np.linspace(0,1.1,12):
    x = [0.01,0.1,0.49,0.51,0.9,0.99]
    v = []
    for i in x:
        for j in range(1000):
            i = update(i,w=0.5)
        v.append(i)
    plt.grid()
    plt.plot(x,v,label=str(b)[:4],color=(1.4-b,1.4-b,1.4-b),linewidth=2)
    if b == 1:
        plt.annotate(r"$b=1$",(0.9,0.8))
    if b == 1.1:
        plt.annotate(r"$b>1$",(0.9,1.05)) 
    if b == 0.6:
        plt.annotate(r"$b<1$",(0.9,0.4))
    #print(v[-2])
# plt.legend(title="Assimilation bias")
sns.despine()
plt.xlabel("Initial opinion")
plt.ylabel("Final opinion")
plt.title("(A): No personalization")
plt.ylim(0,1.1)
plt.xlim(0,1.05)

plt.subplot(1,2,2)
for b in [0.6,0.7,0.8,0.9,1,1.1]:# np.linspace(0,1.1,12):
    x = [0.01,0.1,0.49,0.51,0.9,0.99]
    v = []
    for i in x:
        for j in range(1000):
            i = update(i)
        v.append(i)
    plt.grid()
    plt.plot(x,v,label=str(b)[:3],color=(1.4-b,1.4-b,1.4-b),linewidth=2)

    #print(v[-2])
# plt.legend(loc=4)
# plt.plot([0.82,0.82],[0.08,0.67],color="gray")
# plt.annotate(r"$b$",(0.78,0.4))

plt.annotate(r"$b=0.6$",(1.,0.65))
plt.annotate(r"$b=0.7$",(1,0.8))
plt.annotate(r"$b=0.8$",(1,0.93))
plt.annotate(r"$b>0.8$",(1,1.05))

sns.despine()
plt.xlabel("Initial opinion")
plt.ylabel("Final opinion")
plt.title("(B): Personalization")
plt.ylim(0,1.1)
plt.xlim(0,1.15)

plt.tight_layout()
plt.savefig("figures/sim_polarization.pdf")
plt.show()

